# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [87]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import random

BATCH_SIZE = 128
NUM_EPOCHS = 10

seed = 1337
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [65]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [99]:
import torch.nn.functional as F
import torch.optim as optim

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.35)
        self.fc1 = nn.Linear(16*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        #x = self.dropout(x)
        x = F.relu(self.fc2(x))
        #x = self.dropout(x)
        x = self.fc3(x)
        return x

    
model = SimpleNet()
print(model)

# TODO:define loss function and optimiter
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=5e-2, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1000,gamma = 0.8)

SimpleNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.35, inplace=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Next, we can start to train and evaluate!

In [ ]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    time.sleep(0.5)
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    print("Evaluating...")
    model.eval()
    train_acc = 0
    for images, labels in train_loader:
        output = model(images)
        pred = torch.max(output, 1)[1]
        train_correct = (pred == labels).sum()
        train_acc += train_correct
        
    print("Epoch " , epoch)
    print("Train Accuracy：%0.2f%%" % (train_acc/600))
    
    test_acc = 0
    for images, labels in test_loader:
        output = model(images)
        pred = torch.max(output, 1)[1]
        test_correct = (pred == labels).sum()
        test_acc += test_correct
    print("Test Accuracy：%0.2f%%" % (test_acc/100))

100%|██████████| 468/468 [00:15<00:00, 30.11it/s]


Evaluating...
Epoch  0
Train Accuracy：97.60%
Test Accuracy：97.66%


100%|██████████| 468/468 [00:14<00:00, 32.53it/s]


Evaluating...
Epoch  1
Train Accuracy：98.52%
Test Accuracy：98.32%


100%|██████████| 468/468 [00:14<00:00, 32.71it/s]


...

100%|██████████| 468/468 [00:14<00:00, 31.75it/s]


Evaluating...
Epoch  7
Train Accuracy：99.70%
Test Accuracy：99.03%


100%|██████████| 468/468 [00:14<00:00, 33.20it/s]


Evaluating...
Epoch  8
Train Accuracy：99.76%
Test Accuracy：98.95%


 48%|████▊     | 224/468 [00:06<00:07, 33.70it/s]

#### Q5:
Please print the training and testing accuracy.